In [ ]:
import pandas as pd

import re

import nltk
from nltk.tokenize.toktok import ToktokTokenizer

from nltk.tokenize.toktok import ToktokTokenizer

from tqdm.notebook import tqdm

from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer
tqdm.pandas()

import spacy

from keras.models import Sequential
from keras.layers import Dropout, Conv1D, Flatten, Dense, Embedding
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorboard.plugins import projector
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split

In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/odedovadia/AdvancedMLDLCourse/main/Assignments/Assignment%20II%20-%20NLP/train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/odedovadia/AdvancedMLDLCourse/main/Assignments/Assignment%20II%20-%20NLP/test.csv')

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
train_df['Show'].value_counts()

## Preprocessing

### Text cleaning

In [ ]:
# Define function for removing special characters
def remove_special_characters(text):
    pattern = r'[^a-zA-Z0-9\s]'
    text = re.sub(pattern,'',text)
    return text

train_df['Dialogue'] = train_df['Dialogue'].apply(remove_special_characters)
test_df['Dialogue'] = test_df['Dialogue'].apply(remove_special_characters)

In [ ]:
sentence_1 = train_df['Dialogue'][1]
sentence_1

### Removing stop words

In [ ]:
tokenizer = ToktokTokenizer()

# Create stop word list
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')

In [ ]:
# Removing the stopwords

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

# Apply function on review column
train_df['Dialogue'] = train_df['Dialogue'].apply(remove_stopwords)
test_df['Dialogue'] = test_df['Dialogue'].apply(remove_stopwords)

In [ ]:
sentence_2 = train_df['Dialogue'][1]
print('before:', sentence_1,'\n','after:', train_df['Dialogue'][1])

### Lemmatizer



In [ ]:
# Download required NLTK resources
nltk.download('punkt')
nltk.download('wordnet')

# Initialize WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function for lemmatization
def lemmatize_sentence(sentence):
    # Tokenize the sentence into words
    words = word_tokenize(sentence)

    # Lemmatize each word in the sentence
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # Join the lemmatized words back into a sentence
    lemmatized_sentence = ' '.join(lemmatized_words)

    return lemmatized_sentence

# Apply lemmatization to each sentence in the DataFrame
train_df['Lemmatized_Dialogue'] = train_df['Dialogue'].apply(lemmatize_sentence)
test_df['Lemmatized_Dialogue'] = test_df['Dialogue'].apply(lemmatize_sentence)
# Display the DataFrame with lemmatized dialogue
print(train_df[['Dialogue', 'Lemmatized_Dialogue']].head())


In [ ]:
print(train_df['Lemmatized_Dialogue'][3])
print(train_df['Dialogue'][3])

### Train-test-valdiation split

In [ ]:
X = train_df['Lemmatized_Dialogue']
Y = train_df['Show']

x_test = test_df['Lemmatized_Dialogue']
y_test = test_df['Show']

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=42)

Check that the distribution of our labels between test and train is not skewed toward one class:

In [ ]:
train_label_count = y_train.value_counts()
test_label_count = y_test.value_counts()

print('Train negative to positive ratio:', train_label_count['Friends'] / train_label_count['Seinfeld'])
print('Test negative to positive ratio:', test_label_count['Friends'] / test_label_count['Seinfeld'])

In [ ]:
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)
print('x_val shape:', x_val.shape)
print('y_val shape:', y_val.shape)

Convert 'Friends' to 0 and 'Seinfeld' to 1

In [ ]:
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)
y_val = lb.transform(y_val)

## Modeling

In [ ]:
token = Tokenizer(lower=False)
token.fit_on_texts(x_train)

Convert to sequences:

In [ ]:
x_train_dl = token.texts_to_sequences(x_train)
x_val_dl = token.texts_to_sequences(x_val)
x_test_dl = token.texts_to_sequences(x_test)

In [ ]:
x_train_dl[1]

Calculate document lengths:

In [ ]:
doc_len_list = [len(doc) for doc in x_train_dl]

Longest, shortest, and average document:

In [ ]:
max(doc_len_list), min(doc_len_list), np.mean(doc_len_list)

Let's examine the distribution:

In [ ]:
sns.histplot(doc_len_list)

50 seems like a reasonable shared length of words

In [ ]:
max_words = 50

x_train_dl = sequence.pad_sequences(x_train_dl, maxlen=max_words)
x_val_dl = sequence.pad_sequences(x_val_dl, maxlen=max_words)
x_test_dl = sequence.pad_sequences(x_test_dl, maxlen=max_words)

Let's see what a sample look like:

In [ ]:
total_words = len(token.word_index) + 1
total_words

In [ ]:
x_train_dl.shape

In [ ]:
x_train_dl[10, :]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

model = Sequential()

# Embedding Layer
model.add(Embedding(input_dim=total_words, output_dim=32, input_length=max_words))

# Bidirectional LSTM Layer (Return Sequences)
model.add(Bidirectional(LSTM(50, return_sequences=True)))

# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(50)))

model.add(Dropout(0.6))  # Adjust the dropout rate as needed

# Dense Layer
model.add(Dense(1, activation='sigmoid'))

model.compile(
     loss='binary_crossentropy',
     optimizer='adam',
     metrics=['accuracy'])
model.summary()

We will train the model with a maximum of 20 epochs, and we'll set a condition to stop epoching when the accuracy no longer improves.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

In [ ]:
model.fit(x_train_dl, y_train, validation_data=(x_val_dl, y_val), epochs=20, batch_size=32, callbacks=[early_stopping])

Epoch 1/20
1981/1981 [==============================] - 234s 114ms/step - loss: 0.5287 - accuracy: 0.7029 - val_loss: 0.4815 - val_accuracy: 0.7365
Epoch 2/20
 901/1981 [============>.................] - ETA: 1:52 - loss: 0.4237 - accuracy: 0.7832

Let's evaluate our model:

In [ ]:
model.evaluate(x_test_dl, y_test)
train_acc = model.evaluate(x_train_dl, y_train)[1]
model.evaluate(x_val_dl, y_val)

## Exploration

In [ ]:
#Characters name by show
characters_by_show_df = test_df.groupby('Show')['Character'].unique().reset_index()
characters_by_show_df.columns = ['Show', 'Characters']
characters_by_show_df

In [ ]:
test_df

In [ ]:
# Get the predicted probabilities for each class
predicted_probabilities = model.predict(x_test_dl)

# Convert the predicted probabilities to predicted class labels
predicted_classes = (predicted_probabilities > 0.5).astype(int)  # Assuming binary classification with threshold 0.5

# Assuming that '1' corresponds to 'Friends' and '0' corresponds to 'Seinfeld'
predicted_shows = ['Friends' if pred == 0 else 'Seinfeld' for pred in predicted_classes]

In [ ]:
# Define a function to count correct and incorrect classifications for each character
def count_classification_errors(actual, predicted):
    correct = sum(1 for a, p in zip(actual, predicted) if a == p)
    incorrect = len(actual) - correct
    return correct, incorrect

In [ ]:
def process_show(characters, show_name, actuals, predictions):
    correct_counts = []
    incorrect_counts = []
    correct_percentages = []
    incorrect_percentages = []

    for character in characters:
        mask = (test_df['Character'] == character) & (test_df['Show'] == show_name)
        character_actuals = actuals[mask]
        character_predictions = [predictions[i] for i, m in enumerate(mask) if m]
        correct, incorrect = count_classification_errors(character_actuals, character_predictions)

        total = correct + incorrect
        correct_counts.append(correct)
        incorrect_counts.append(incorrect)
        correct_percentages.append((correct / total) * 100)
        incorrect_percentages.append((incorrect / total) * 100)

    return correct_counts, incorrect_counts, correct_percentages, incorrect_percentages

In [ ]:
actual_shows = test_df['Show']

# Process Friends show
friends_correct_counts, friends_incorrect_counts, friends_correct_percentages, friends_incorrect_percentages = process_show(characters_by_show_df['Characters'][0], 'Friends', actual_shows, predicted_shows)

# Process Seinfeld show
seinfeld_correct_counts, seinfeld_incorrect_counts, seinfeld_correct_percentages, seinfeld_incorrect_percentages = process_show(characters_by_show_df['Characters'][1], 'Seinfeld', actual_shows, predicted_shows)

In [ ]:
def plot_show_statistics(show_name, correct_counts, incorrect_counts, correct_percentages, incorrect_percentages, characters):
    plt.figure(figsize=(12, 8))
    bar_width = 0.4
    indices = range(len(characters))
    plt.bar(indices, correct_counts, width=bar_width, color='green', label=f'{show_name} - Correct')
    plt.bar(indices, incorrect_counts, bottom=correct_counts, width=bar_width, color='red', label=f'{show_name} - Incorrect')
    plt.xlabel('Character')
    plt.ylabel('Count')
    plt.title(f'Correct vs Incorrect Classifications for {show_name} Characters')
    plt.xticks(indices, characters, ha='right')
    plt.legend()

    # Add percentage labels
    for i, (correct, incorrect) in enumerate(zip(correct_percentages, incorrect_percentages)):
        plt.text(i, correct_counts[i] + 5, f'{correct:.1f}%', ha='center', color='black')
        plt.text(i, correct_counts[i] + incorrect_counts[i] + 5, f'{incorrect:.1f}%', ha='center', color='black')

    # Print accuracies for each character
    print(f'Accuracies for {show_name} Characters:')
    for character, correct, incorrect in zip(characters, correct_counts, incorrect_counts):
        total = correct + incorrect
        accuracy = correct / total if total > 0 else 0.0
        print(f'{character}: {accuracy:.2f}')

    plt.tight_layout()
    plt.show()

In [ ]:
# Plot statistics for Friends
plot_show_statistics('Friends', friends_correct_counts, friends_incorrect_counts, friends_correct_percentages, friends_incorrect_percentages, characters_by_show_df['Characters'][0])

In [ ]:
# Plot statistics for Seinfeld
plot_show_statistics('Seinfeld', seinfeld_correct_counts, seinfeld_incorrect_counts, seinfeld_correct_percentages, seinfeld_incorrect_percentages, characters_by_show_df['Characters'][1])

## Report

In [ ]:
loss, accuracy = model.evaluate(x_test_dl, y_test)

In [ ]:
accuracy

In [ ]:
results = {'Test set accuracy': [accuracy],
           'Train set accuracy': [train_acc],
           'Number of trainable parameters': '885893',
           'Number of layers': '5',
           'Regularization methods': 'Dropout',
           'Number of epochs': '4',
           'Choice of loss function': 'binary_crossentropy',
           'Choice of optimizer': 'adam',
           'Embedding dimension': '32'
           }

In [ ]:
df_results = pd.DataFrame(results)
df_results

In [ ]:
# df_results to CSV
import os
df_results.to_csv(os.path.join(os.getcwd(), 'exercise3.csv'))